<a href="https://colab.research.google.com/github/jblcky/retail-inventory-forecasting-2/blob/main/notebooks/model_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading Data from github

In [22]:
import pandas as pd

url = 'https://raw.githubusercontent.com/jblcky/retail-inventory-forecasting-2/refs/heads/main/data/raw/sales_data.csv'
sales_df = pd.read_csv(url, parse_dates=["date"])

sales_df.head()



,date,sku_id,store_id,quantity_sold
0,2024-01-01,SKU_001,Store_A,45
1,2024-01-02,SKU_001,Store_A,51
2,2024-01-03,SKU_001,Store_A,51
3,2024-01-04,SKU_001,Store_A,57
4,2024-01-05,SKU_001,Store_A,46


In [23]:
url = 'https://raw.githubusercontent.com/jblcky/retail-inventory-forecasting-2/refs/heads/main/data/raw/events_calendar.csv'
events_df = pd.read_csv(url, parse_dates=["date"])
events_df.head()

,date,event
0,2024-01-01,Public Holiday
1,2024-01-15,Public Holiday
2,2024-01-29,Flu Season
3,2024-02-12,Public Holiday
4,2024-02-26,Public Holiday


In [24]:
url = 'https://raw.githubusercontent.com/jblcky/retail-inventory-forecasting-2/refs/heads/main/data/raw/inventory_levels.csv'
inventory_df = pd.read_csv(url, parse_dates=['date'])
inventory_df.head()

,date,sku_id,store_id,inventory_level
0,2024-01-01,SKU_001,Store_A,77
1,2024-01-02,SKU_001,Store_A,180
2,2024-01-03,SKU_001,Store_A,124
3,2024-01-04,SKU_001,Store_A,127
4,2024-01-05,SKU_001,Store_A,139


In [25]:
url = 'https://raw.githubusercontent.com/jblcky/retail-inventory-forecasting-2/refs/heads/main/data/raw/sku_metadata.csv'
sku_df = pd.read_csv(url)
sku_df.head()

,sku_id,category,supplier,lead_time_days,unit_cost
0,SKU_001,Medicine,Supplier_1,7,4.53
1,SKU_002,Supplement,Supplier_1,21,7.14
2,SKU_003,Supplement,Supplier_2,7,4.74
3,SKU_004,Medicine,Supplier_4,7,4.58
4,SKU_005,Personal Care,Supplier_2,14,9.47


This step ensures your data is:

Valid and consistent

Free of duplicates or nulls

Well-understood before forecasting

**Data cleaning steps**
- inspect basic information and missing values
- Check nulls
- Check duplicates
- Fix dtypes if needed

In [26]:
# Check shapes
print("Sales shape:", sales_df.shape)
print("Inventory shape:", inventory_df.shape)
print("Events shape:", events_df.shape)
print("SKU Meta shape:", sku_df.shape)

# Check missing values
print("\nMissing values in each dataframe:")
print("Sales:\n", sales_df.isnull().sum())
print("Inventory:\n", inventory_df.isnull().sum())
print("Events:\n", events_df.isnull().sum())
print("SKU Meta:\n", sku_df.isnull().sum())

# Check data types
print("\nData types:")
print(sales_df.dtypes)


Sales shape: (3600, 4)
Inventory shape: (3600, 4)
Events shape: (10, 2)
SKU Meta shape: (10, 5)

Missing values in each dataframe:
Sales:
 date             0
sku_id           0
store_id         0
quantity_sold    0
dtype: int64
Inventory:
 date               0
sku_id             0
store_id           0
inventory_level    0
dtype: int64
Events:
 date     0
event    0
dtype: int64
SKU Meta:
 sku_id            0
category          0
supplier          0
lead_time_days    0
unit_cost         0
dtype: int64

Data types:
date             datetime64[ns]
sku_id                   object
store_id                 object
quantity_sold             int64
dtype: object


basic stats and duplicates

In [27]:
# Basic summary stats
print("\nSales quantity summary:")
display(sales_df['quantity_sold'].describe())

# Check for duplicates
print("\nDuplicates:")
print("Sales:", sales_df.duplicated().sum())
print("Inventory:", inventory_df.duplicated().sum())
print("Events:", events_df.duplicated().sum())
print("SKU Meta:", sku_df.duplicated().sum())



Sales quantity summary:


,quantity_sold
count,3600.000000
mean,30.744444
std,15.206232
min,0.000000
25%,19.000000
50%,28.000000
75%,42.000000
max,71.000000



Duplicates:
Sales: 0
Inventory: 0
Events: 0
SKU Meta: 0


explore unique values

In [28]:
print("Unique SKUs:", sales_df['sku_id'].nunique())
print("Unique Stores:", sales_df['store_id'].nunique())
print("Date range:", sales_df['date'].min(), "to", sales_df['date'].max())

# Peek at top selling SKUs
top_skus = sales_df.groupby('sku_id')['quantity_sold'].sum().sort_values(ascending=False)
print("\nTop selling SKUs:")
display(top_skus.head(5))


Unique SKUs: 10
Unique Stores: 2
Date range: 2024-01-01 00:00:00 to 2024-06-28 00:00:00

Top selling SKUs:


,quantity_sold
sku_id,
SKU_001,17697
SKU_010,14784
SKU_003,13815
SKU_008,12679
SKU_009,11185


In [29]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


**Feature Engineering**
- Our goal now is to enrich the dataset with meaningful features to improve demand forecasting accuracy.

**Feature engineering**
- Time features (day, week, month, etc.)
- Event flags (holidays, flu season, etc.)
- Rolling averages (7d, 14d, etc.)(past 7-day average sales, etc.)
- Merge SKU metadata (optional)

add time feature

In [30]:
sales_df['day_of_week'] = sales_df['date'].dt.dayofweek  # 0 = Monday
sales_df['weekofyear'] = sales_df['date'].dt.isocalendar().week
sales_df['month'] = sales_df['date'].dt.month
sales_df['is_weekend'] = sales_df['day_of_week'] >= 5


Merge Events (Holidays, School Holidays, Flu)
- We’ll mark special days that might affect demand.

In [31]:
# Merge event flags (left join on date)
sales_df = sales_df.merge(events_df, on='date', how='left')

# One-hot encode event type
sales_df = pd.get_dummies(sales_df, columns=['event'], prefix='event', dummy_na=False)

# Fill NaNs with 0 for no event
event_cols = [col for col in sales_df.columns if col.startswith("event_")]
sales_df[event_cols] = sales_df[event_cols].fillna(0)


Add Rolling Sales Features. This captures short-term trends.

In [32]:
# Sort to ensure correct rolling
sales_df = sales_df.sort_values(['sku_id', 'store_id', 'date'])

# 7-day rolling average of past sales per SKU+Store
sales_df['rolling_7d_sales'] = (
    sales_df.groupby(['sku_id', 'store_id'])['quantity_sold']
    .transform(lambda x: x.shift(1).rolling(window=7).mean())
)

# Fill NaNs from early days
sales_df['rolling_7d_sales'] = sales_df['rolling_7d_sales'].fillna(0)


upload csv file to github

In [33]:
sales_df.columns

Index(['date', 'sku_id', 'store_id', 'quantity_sold', 'day_of_week',
       'weekofyear', 'month', 'is_weekend', 'event_Flu Season',
       'event_Public Holiday', 'event_School Holiday', 'rolling_7d_sales'],
      dtype='object')

Add on Feature Engineering
- Rename Event Columns for Consistency
- Why: Shorter names keep things readable and consistent for modeling and documentation.

In [34]:
# Rename event columns to clean, consistent names
sales_df = sales_df.rename(columns={
    'event_Flu Season': 'is_flu_season',
    'event_Public Holiday': 'is_holiday',
    'event_School Holiday': 'is_school_holiday'
})


Convert date Column to datetime and Extract Year
- Why: Extracting the year allows the model to capture temporal shifts like pandemic years, regulatory changes, etc.

In [35]:
# Convert 'date' to datetime if not already
sales_df['date'] = pd.to_datetime(sales_df['date'])

# Extract year for seasonality (e.g., COVID year, annual effects)
sales_df['year'] = sales_df['date'].dt.year


Add Synthetic Pricing (If Missing)
- Why: Price fluctuations impact demand. This synthetic price adds realism if actual prices aren’t available.

In [36]:
# Add random but realistic prices if missing
if 'price' not in sales_df.columns:
    import numpy as np
    np.random.seed(42)  # For reproducibility
    sales_df['price'] = np.random.uniform(5.0, 35.0, size=len(sales_df))


Add Dummy Promotion Flag
- Why: Promotions typically spike demand. This dummy column preps the model structure so you can integrate real promo data later.

In [37]:
# Add a placeholder promotion column if not provided
sales_df['on_promo'] = 0


Add Days to Expiry (Placeholder for Now)
-  Why: This allows future integration with expiry-aware redistribution logic. For now, it ensures consistent feature shape.

In [38]:
# Add placeholder for days to expiry (for Project B compatibility)
sales_df['days_to_expiry'] = 999


Final Feature List for Model Training
- Why: Centralizing the features list makes the model training pipeline easier to maintain and debug.

In [39]:
# Define final set of features and target variable
features = [
    'store_id', 'sku_id', 'day_of_week', 'weekofyear', 'month', 'year',
    'is_weekend', 'is_holiday', 'is_school_holiday', 'is_flu_season',
    'price', 'on_promo', 'days_to_expiry', 'rolling_7d_sales'
]

target = 'quantity_sold'


In [40]:
sales_df.head(20)

,date,sku_id,store_id,quantity_sold,day_of_week,weekofyear,month,is_weekend,is_flu_season,is_holiday,is_school_holiday,rolling_7d_sales,year,price,on_promo,days_to_expiry
0,2024-01-01,SKU_001,Store_A,45,0,1,1,False,False,True,False,0.000000,2024,16.236204,0,999
1,2024-01-02,SKU_001,Store_A,51,1,1,1,False,False,False,False,0.000000,2024,33.521429,0,999
2,2024-01-03,SKU_001,Store_A,51,2,1,1,False,False,False,False,0.000000,2024,26.959818,0,999
3,2024-01-04,SKU_001,Store_A,57,3,1,1,False,False,False,False,0.000000,2024,22.959755,0,999
4,2024-01-05,SKU_001,Store_A,46,4,1,1,False,False,False,False,0.000000,2024,9.680559,0,999
5,2024-01-06,SKU_001,Store_A,51,5,1,1,True,False,False,False,0.000000,2024,9.679836,0,999
6,2024-01-07,SKU_001,Store_A,42,6,1,1,True,False,False,False,0.000000,2024,6.742508,0,999
7,2024-01-08,SKU_001,Store_A,50,0,2,1,False,False,False,False,49.000000,2024,30.985284,0,999
8,2024-01-09,SKU_001,Store_A,55,1,2,1,False,False,False,False,49.714286,2024,23.033450,0,999
9,2024-01-10,SKU_001,Store_A,49,2,2,1,False,False,False,False,50.285714,2024,26.242177,0,999


In [41]:
sales_df.tail(20)

,date,sku_id,store_id,quantity_sold,day_of_week,weekofyear,month,is_weekend,is_flu_season,is_holiday,is_school_holiday,rolling_7d_sales,year,price,on_promo,days_to_expiry
3580,2024-06-09,SKU_010,Store_B,48,6,23,6,True,False,False,False,55.571429,2024,19.495426,0,999
3581,2024-06-10,SKU_010,Store_B,50,0,24,6,False,False,False,False,55.142857,2024,30.146913,0,999
3582,2024-06-11,SKU_010,Store_B,55,1,24,6,False,False,False,False,54.714286,2024,15.842113,0,999
3583,2024-06-12,SKU_010,Store_B,51,2,24,6,False,False,False,False,54.857143,2024,30.793840,0,999
3584,2024-06-13,SKU_010,Store_B,53,3,24,6,False,False,False,False,53.857143,2024,17.204144,0,999
3585,2024-06-14,SKU_010,Store_B,51,4,24,6,False,False,False,False,53.285714,2024,14.846993,0,999
3586,2024-06-15,SKU_010,Store_B,58,5,24,6,True,False,False,False,51.571429,2024,18.619697,0,999
3587,2024-06-16,SKU_010,Store_B,50,6,24,6,True,False,False,False,52.285714,2024,27.873409,0,999
3588,2024-06-17,SKU_010,Store_B,50,0,25,6,False,False,False,False,52.571429,2024,8.780061,0,999
3589,2024-06-18,SKU_010,Store_B,55,1,25,6,False,False,False,False,52.571429,2024,10.892576,0,999


In [42]:
# Save to CSV
sales_df.to_csv("sales_df_fe.csv", index=False)
